<font color='blue'><h1>Introdução Prática ao PyTorch</h1></font>

<font color='0471A6'><h2>Loop de Otimização no PyTorch - Exemplo Simples Multi-Task</h2></font>

- O modelo é treinado / otimizado em um loop de otimização
- Cada iteração será uma época
- Cada época contém duas partes principais:
  - Loop de treinamento
    - iteração sobre o conjunto de treinamento
  - Loop de validação
    - iteração sobre o conjunto de validação

Hiperparâmetros (parâmetros ajustáveis pelo projetista). Exemplos:
- Número de Épocas
- Tamanho do Lote (Batch)
- Taxa de Aprendizagem

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test\
= train_test_split(X, y, test_size=0.25)

X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 8)

        self.fc3a = nn.Linear(8, 4)
        self.fc4a = nn.Linear(4, 1)

        self.fc3b = nn.Linear(8, 4)
        self.fc4b = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        h = self.sigmoid(self.fc2(x))

        xa = self.sigmoid(self.fc3a(h))
        xa = self.sigmoid(self.fc4a(xa))


        xb = self.sigmoid(self.fc3b(h))
        xb = self.sigmoid(self.fc4b(xb))

        return xa,xb

net = MLP()
criterion1, criterion2 = nn.MSELoss(), nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
net.train()

# Registro hooks para imprimir cada parametro
def print_grad(name):
    def hook(grad):
        print(f"{name}")
    return hook

for name, param in net.named_parameters():
    param.register_hook(print_grad(name))

# Treinando a rede neural
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        xa, xb = net(inputs)
        loss1 = criterion1(xa, labels.view(-1, 1))
        loss2 = criterion2(xb, labels.view(-1, 1))
        loss = loss1 + loss2

        loss.backward()
        break
        optimizer.step()

        running_loss += loss.item()
        break

    if epoch % 100 == 0:
      print('Epoch %d loss: %.3f' % (epoch,
          running_loss / len(train_loader)))
    break

fc4b.bias
fc4b.weight
fc3b.bias
fc3b.weight
fc4a.bias
fc4a.weight
fc3a.bias
fc3a.weight
fc2.bias
fc2.weight
fc1.bias
fc1.weight
Epoch 0 loss: 0.000


In [2]:
net.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        xa, xb = net(inputs)
        predictions = torch.round(xa)
        total += targets.size(0)
        correct += (predictions ==
                    targets.unsqueeze(1)).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.4f}")

Accuracy on test set: 0.6364


In [3]:
# loss.backward(inputs=list(net.fc2.parameters()))
# optimizer.step()
#for name, param in net.named_parameters():
#    pass # print(name, param.grad)